In [1]:
using JSON
using JuMP, MathOptInterface, Cbc
const MOI = MathOptInterface

const ztol = 1e-6
const inf = 1e20

1.0e20

In [2]:
INTERVENTIONS = "Interventions"
NUM_SCENARIO= "Scenarios_number"
RESOURCES = "Resources"
SEASONS = "Seasons"
EXCLUSIONS = "Exclusions"
QUANTILE = "Quantile"
ALPHA = "Alpha"

"Alpha"

In [3]:
parsed_file = JSON.parsefile("example2.json")
keys(parsed_file)

Base.KeySet for a Dict{String,Any} with 8 entries. Keys:
  "Seasons"
  "Quantile"
  "T"
  "Interventions"
  "Exclusions"
  "Resources"
  "Scenarios_number"
  "Alpha"

In [4]:
get(parsed_file, EXCLUSIONS, [])

Dict{String,Any} with 1 entry:
  "E1" => Any["I2", "I3", "full"]

In [5]:
get(get(parsed_file, INTERVENTIONS, []), "I2", [])

Dict{String,Any} with 4 entries:
  "tmax"     => 3
  "risk"     => Dict{String,Any}("1"=>Dict{String,Any}("1"=>Any[8],"2"=>Any[0],…
  "Delta"    => Any[1, 1, 1]
  "workload" => Dict{String,Any}("c1"=>Dict{String,Any}("1"=>Dict{String,Any}("…

In [6]:
get(get(parsed_file, INTERVENTIONS, []), "I3", [])

Dict{String,Any} with 4 entries:
  "tmax"     => 2
  "risk"     => Dict{String,Any}("1"=>Dict{String,Any}("1"=>Any[2],"2"=>Any[0])…
  "Delta"    => Any[1, 1, 2]
  "workload" => Dict{String,Any}("c1"=>Dict{String,Any}("1"=>Dict{String,Any}("…

In [7]:
#Constants
n = length(keys(get(parsed_file, INTERVENTIONS, [])))
C = length(keys(get(parsed_file, RESOURCES, [])))
T = get(parsed_file, "T", 0)
St = get(parsed_file, NUM_SCENARIO, 0)
S = max(St...)
tau = get(parsed_file, QUANTILE, 0)
alpha = get(parsed_file, ALPHA, 0)

0.5

In [8]:
risk_arr = zeros(n, T, T, S) #(intervention, current time, start time, scenario)
#Populate risk_arr
for (intervention, int_values) in pairs(get(parsed_file, INTERVENTIONS, []))
    for (curtime, ctime_values) in pairs(get(int_values, "risk", []))
        for (start_time, stime_values) in pairs(ctime_values)
            for s in 1:getindex(St, parse(Int64, curtime))
                setindex!(risk_arr, stime_values[s], parse(Int64, SubString(intervention, 2)), parse(Int64, curtime), parse(Int64, start_time), s)
            end
        end
    end
end
M = max(risk_arr...) + 1
println(M)
print(risk_arr)

21.0
[7.0 4.0 2.0; 8.0 0.0 0.0; 2.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 5.0 0.0; 0.0 6.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 6.0; 0.0 0.0 0.0]

[0.0 0.0 20.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 8.0; 0.0 0.0 0.0]

In [9]:
r_arr = zeros(n, T, T, C) #(intervention, current time, start time, resource)
#Populate r_arr
for (intervention, int_values) in pairs(get(parsed_file, INTERVENTIONS, []))
    for (resource, resource_values) in pairs(get(int_values, "workload", []))
        for (curr_time, curr_time_values) in pairs(resource_values)
            for (start_time, start_time_load) in pairs(curr_time_values)
                println(start_time_load)
                setindex!(r_arr, start_time_load, parse(Int64, SubString(intervention, 2)), parse(Int64, curr_time), parse(Int64, start_time), parse(Int64, SubString(resource, 2)))
            end
        end
    end
end

31
0
8
14
0
0
0
14
0
0
0
14
5
0
0
5
0
0


In [10]:
u = zeros(C, T)
for (res, res_values) in pairs(get(parsed_file, RESOURCES, []))
    setindex!(u, get(res_values, "max", []), parse(Int64, SubString(res, 2)), 1:T)
end
print(u)

[49.0 23.0 15.0]

In [11]:
l = zeros(C, T)
for (res, res_values) in pairs(get(parsed_file, RESOURCES, []))
    setindex!(l, get(res_values, "min", []), parse(Int64, SubString(res, 2)), 1:T)
end
print(l)

[10.0 0.0 6.0]

In [12]:
deltas = zeros(n, T)
for (intervention, int_values) in pairs(get(parsed_file, INTERVENTIONS, []))
    setindex!(deltas, get(int_values, "Delta", []), parse(Int64, SubString(intervention, 2)), 1:T)
end
print(deltas)

[3.0 3.0 2.0; 1.0 1.0 1.0; 1.0 1.0 2.0]

In [13]:
r_arr

3×3×3×1 Array{Float64,4}:
[:, :, 1, 1] =
 31.0  0.0  8.0
 14.0  0.0  0.0
  5.0  0.0  0.0

[:, :, 2, 1] =
 0.0   0.0  0.0
 0.0  14.0  0.0
 0.0   5.0  0.0

[:, :, 3, 1] =
 0.0  0.0   0.0
 0.0  0.0  14.0
 0.0  0.0   0.0

In [14]:
#Construct JuMP Model 
j_model = Model()
t[1:n] = @variable(j_model, t[1:n], Int)
d[1:n] = @variable(j_model, d[1:n], Int)
y[1:n, 1:T] = @variable(j_model, y[1:n, 1:T], Bin)
z1[1:n, 1:T] = @variable(j_model, z1[1:n, 1:T], Bin)
z2[1:n, 1:T] = @variable(j_model, z2[1:n, 1:T], Bin)
a[1:n, 1:T] = @variable(j_model, a[1:n, 1:T], Bin)
c[1:n, 1:T, 1:T] = @variable(j_model, c[1:n, 1:T, 1:T], Bin) #of the form c[intervention, current time, start time]
risk[1:S, 1:T] = @variable(j_model, risk[1:S, 1:T])
w[1:S, 1:T] = @variable(j_model, w[1:S, 1:T], Bin)
q[1:T] = @variable(j_model, q[1:T])
e[1:T] = @variable(j_model, e[1:T])
mean_risk_t = [sum(risk[s, t1] for s in 1:getindex(St, t1))/getindex(St, t1) for t1 in 1:T]
mean_risk = sum(getindex(mean_risk_t, t1) for t1 in 1:T)/T
quant_risk = [q[t1] - getindex(mean_risk_t, t1) for t1 in 1:T]

constr1 = [@constraint(j_model, t[i] >= time*y[i,time]) for i in 1:n for time in 1:T]
constr2 = [@constraint(j_model, t[i] <= time + T*(1-y[i, time])) for i in 1:n for time in 1:T]
constr3a = [@constraint(j_model, z1[i, t1] <= sum(y[i, j] for j in 1:t1) + ztol)  for i in 1:n for t1 in 1:T]
constr3b = [@constraint(j_model, z1[i, t1] >= sum(y[i, j] for j in 1:t1) - ztol)  for i in 1:n for t1 in 1:T]
constr4 = [@constraint(j_model, d[i] >= t1*z2[i, t1]) for i in 1:n for t1 in 1:T]
constr5 = [@constraint(j_model, d[i] <= t1 + (T+1)*z2[i, t1]) for i in 1:n for t1 in 1:T]
constr6 = [@constraint(j_model, a[i, t1] >= z1[i, t1] + z2[i, t1] - 1) for i in 1:n for t1 in 1:T]
constr7 = [@constraint(j_model, a[i, t1] <= z1[i, t1]) for i in 1:n for t1 in 1:T]
constr8 = [@constraint(j_model, a[i, t1] <= z2[i, t1]) for i in 1:n for t1 in 1:T]                        
constr9 = [@constraint(j_model, c[i, time1, time2] >= y[i, time2] + a[i, time1] - 1) for i in 1:n for time1 in 1:T for time2 in 1:T]
constr10 = [@constraint(j_model, c[i, time1, time2] <= y[i, time2]) for i in 1:n for time1 in 1:T for time2 in 1:T]
constr11 = [@constraint(j_model, c[i, time1, time2] <= a[i, time1]) for i in 1:n for time1 in 1:T for time2 in 1:T]
constr12 = [@constraint(j_model, d[i] <= time + deltas[i, time] + T * (1 - y[i, time])) for i in 1:n for time in 1:T]
constr13 = [@constraint(j_model, d[i] >= time + deltas[i, time] - T*(1 - y[i, time])) for i in 1:n for time in 1:T]
constr14 = [@constraint(j_model, risk[s, t1] == sum(getindex(risk_arr, i, t1, t2, s) * c[i, t1, t2] for i in 1:n for t2 in 1:T)) for t1 in 1:T for s in 1:S]
constr15 = [@constraint(j_model, q[t1] >= risk[s, t1] - M * (1 - w[s, t1])) for s in 1:S for t1 in 1:T]
constr16 = [@constraint(j_model, q[t1] >= 0) for t1 in 1:T]
constr17 = [@constraint(j_model, e[t1] >= 0) for t1 in 1:T]
constr18 = [@constraint(j_model, e[t1] >= getindex(quant_risk, t1)) for t1 in 1:T]                                                                                                            
constr17 = [@constraint(j_model, d[i] <= T+ 1) for i in 1:n] 
constr18 = [@constraint(j_model, d[i] >= 0) for i in 1:n] 
constr19 = [@constraint(j_model, sum(y[i, time] for time in 1:T) == 1) for i in 1:n]
constr20 = [@constraint(j_model, sum(getindex(r_arr, i, t1, t2, cj) * c[i, t1, t2] for i in 1:n for t2 in 1:T) >= l[cj, t1])  for t1 in 1:T for cj in 1:C]
constr21 = [@constraint(j_model, sum(getindex(r_arr, i, t1, t2, cj) * c[i, t1, t2] for i in 1:n for t2 in 1:T) <= u[cj, t1])  for t1 in 1:T for cj in 1:C]
constr23 = [@constraint(j_model, sum(w[s, t1] for s in 1:getindex(St, t1)) >= tau*getindex(St, t1)) for t1 in 1:T]                                                    
constr24 = [@constraint(j_model, t[i] <= T) for i in 1:n]
constr25 = [@constraint(j_model, t[i] >= 0) for i in 1:n]

3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 t[1] ≥ 0.0
 t[2] ≥ 0.0
 t[3] ≥ 0.0

In [15]:
constr20

3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 31 c[1,1,1] + 14 c[2,1,1] + 5 c[3,1,1] ≥ 10.0
 14 c[2,2,2] + 5 c[3,2,2] ≥ 0.0               
 8 c[1,3,1] + 14 c[2,3,3] ≥ 6.0               

In [16]:
#Constraint 22
for (exc, exc_values) in pairs(get(parsed_file, EXCLUSIONS, []))
    i1 = parse(Int64, SubString(getindex(exc_values, 1), 2))
    i2 = parse(Int64, SubString(getindex(exc_values, 2), 2))    
    season = getindex(exc_values, 3)
    season_vals = get(get(parsed_file, SEASONS, []), season, [])
    [@constraint(j_model, a[i1, time] + a[i2, time] <= 1) for time in season_vals]
end

In [17]:
@objective(j_model, Min, alpha * mean_risk + (1 - alpha) * sum(e[t1] for t1 in 1:T)/T)
print(j_model)

Min 0.16666666666666666 risk[1,1] + 0.16666666666666666 risk[1,2] + 0.08333333333333333 risk[1,3] + 0.08333333333333333 risk[2,3] + 0.16666666666666666 e[1] + 0.16666666666666666 e[2] + 0.16666666666666666 e[3]
Subject to
 risk[1,1] - 7 c[1,1,1] - 8 c[2,1,1] - 2 c[3,1,1] = 0.0
 risk[2,1] = 0.0
 risk[1,2] - 4 c[1,2,1] - 5 c[2,2,2] - 6 c[3,2,2] = 0.0
 risk[2,2] = 0.0
 risk[1,3] - 2 c[1,3,1] - 6 c[2,3,3] = 0.0
 risk[2,3] - 20 c[1,3,1] - 8 c[2,3,3] = 0.0
 y[1,1] + y[1,2] + y[1,3] = 1.0
 y[2,1] + y[2,2] + y[2,3] = 1.0
 y[3,1] + y[3,2] + y[3,3] = 1.0
 t[1] - y[1,1] ≥ 0.0
 t[1] - 2 y[1,2] ≥ 0.0
 t[1] - 3 y[1,3] ≥ 0.0
 t[2] - y[2,1] ≥ 0.0
 t[2] - 2 y[2,2] ≥ 0.0
 t[2] - 3 y[2,3] ≥ 0.0
 t[3] - y[3,1] ≥ 0.0
 t[3] - 2 y[3,2] ≥ 0.0
 t[3] - 3 y[3,3] ≥ 0.0
 z1[1,1] - y[1,1] ≥ -1.0e-6
 z1[1,2] - y[1,1] - y[1,2] ≥ -1.0e-6
 z1[1,3] - y[1,1] - y[1,2] - y[1,3] ≥ -1.0e-6
 z1[2,1] - y[2,1] ≥ -1.0e-6
 z1[2,2] - y[2,1] - y[2,2] ≥ -1.0e-6
 z1[2,3] - y[2,1] - y[2,2] - y[2,3] ≥ -1.0e-6
 z1[3,1] - y[3,1] ≥ -1.0e-

In [18]:
set_optimizer(j_model, Cbc.Optimizer)
optimize!(j_model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0.876344 - 0.00 seconds
Cgl0003I 12 fixed, 5 tightened bounds, 40 strengthened rows, 39 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 115 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 4.83333 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds

In [19]:
value.(q)

3-element Array{Float64,1}:
 8.999999999999998
 9.0              
 2.0              

In [20]:
value.(risk[1:S, 3])

2-element Array{Float64,1}:
  2.0              
 19.999999999999996

In [21]:
value.(w[1:S, 1])

2-element Array{Float64,1}:
 1.0
 0.0

In [22]:
value.(y)

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0

In [23]:
value.(t)

3-element Array{Float64,1}:
 1.0
 2.0
 1.0

In [24]:
value.(c[1, 1:3, 1:3])

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0

In [25]:
value.(a[2, 1:3])

3-element Array{Float64,1}:
 0.0
 1.0
 0.0

In [26]:
value.(z1[1, 1:3])

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [27]:
value.(z2[1, 1:3])

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [28]:
deltas

3×3 Array{Float64,2}:
 3.0  3.0  2.0
 1.0  1.0  1.0
 1.0  1.0  2.0

In [29]:
value.(d)

3-element Array{Float64,1}:
 4.0
 3.0
 2.0

In [30]:
value.(t)

3-element Array{Float64,1}:
 1.0
 2.0
 1.0

In [31]:
value.(sum(e[t1] for t1 in 1:T)/T)

0.0

In [32]:
value.(q)

3-element Array{Float64,1}:
 8.999999999999998
 9.0              
 2.0              

In [33]:
value.(mean_risk)

9.666666666666666